# Standardised Approach for Counterparty Credit Risk

Abbreviations: <br >
- $VM$: Variation Margin
- $EAD$: Exposure At Default under SA
- $RC$: replacement cost (different for margined and unmargined)
- $PFE$: Potential Future Exposure
- $V$: Current Market Value of the Derivative in the netting set
- $C$: Net harcut value of the net collateral held
- $ICA$: Independent Collateral Amount
- $TH$: Positive threshold before the counterparty must send the bank collateral
- $MTA$: Minimum transfer amount
- $NICA$: Net independent Collateral Amount
- $MF_i^{type}$: Maturity Factor
- $M_i$: Maturity (i remaining maturity floored by 10 business days)
- $S_i$: Start
- $E_i$: end date
- $T_i$: Contractual Exercise Date
- ${SD}_i$: Supervisory Duration
- $\delta_i$: Supervisory delta Adjustments
- $MPOR_i$: Margin period of risk appropriate for the margin agreement containing the transaction i

Formulas: <br> 
- EAD = alpha * (RC + PFE) 
- RC (Margined) = max{V - C;TH + MTA - NICA; 0}
- PFE = multiplier * $AddOn^{aggregate}$
- multipiler = min{1; Floor + (1 - Floor) * exp ($\frac{V - C}{2* (1-Floor) * AddOn^{aggregate} }$)}
- $AddOn^{aggregate}$ = $\sum_{a} AddOn^a$
- ${SD}_i$ = $\frac {exp(-0.05*S_i)-exp(-0.05*Ei)} {0.05}$
- ${MF}_i^{unmargined}$ = $\sqrt{\frac {min\{M_i; 1 year\}}{1year}}$
- ${MF}_i^{(margined)}$ = $\frac {3} {2} \sqrt{\frac{MPOR_i}{1year}}$

<a href="https://www.bis.org/publ/bcbs279.pdf">PDF Reference</a>

In [60]:
import pandas as pd
import math
import numpy as np

In [61]:
# import reference table for supervisory parameter as supervisory_ref
supervisory_ref = pd.read_csv('supervisory_ref.csv')
supervisory_ref

,Asset Class,Subclass,Supervisory factor,Correlation
0,credit,AAA,0.0038,0.5
1,credit,AA,0.0038,0.5
2,credit,A,0.0042,0.5
3,credit,BBB,0.0054,0.5
4,credit,BB,0.0106,0.5
5,credit,B,0.0160,0.5
6,credit,CCC,0.0600,0.5
7,credit,IG,0.0038,0.8
8,credit,SG,0.0106,0.8


In [350]:
example_1_input = pd.read_csv("example_1_input.csv")
example_1_input

,Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands),Option position,Underlying price,Strike price
0,1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30,NaN,NaN,NaN
1,2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20,NaN,NaN,NaN
2,3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50,Bought,0.06,0.05


In [62]:
#rc(df) takes in a dataframe and compute the RC of a given file
def rc(df):
    v_c = df[['Market value (thousands)']].sum()
    r_c = max(int(v_c), 0)
    return r_c

In [63]:
# nature_classifier(original_df, i) determine the category that the nature
# belongs to, 1 means interest rate, 2 means credit derivative
# 3 means commodity forrward contract
def nature_classifier(original_df, i):
    nature = str(original_df.at[i, 'Nature'])
    if (nature.find('swap') != -1):
        return 1
    elif nature.find('CDS') != -1 or nature.find('Credit') != -1:
        return 2
    else:
        return 3

In [64]:
# convert residual maturity to two lists, S and E by taking in
# three variables, emptyEi, emptySi and residual maturity
def residual_maturity_converter(Ei, Si, residual_maturity):
    for item in residual_maturity:
        item = str(item)
        idx = item.find("into")
        if (idx == -1): # S would be 0
            Si.append(0)
            end = float(item[:item.find("year") - 1])
            Ei.append(end)
        else:
            idx -= 1
            start = int(item[:idx])
            Si.append(start)
            idx += 5
            end = float(item[idx:item.find("year") - 1])
            Ei.append(end)
            
    for i in range(0,len(Ei)):
            Ei[i] += Si[i]
            
    print(Ei, Si, residual_maturity)

In [65]:
#SD_generator takes in Si and Ei and generate the appropriate SD as a list
def SD_generator(Si, Ei, SD):
    if (len(Si) != len(Ei)):
        return "Error"
    for i in range(0, len(Si)):
        si = Si[i]
        ei = Ei[i]
        sd = (math.exp(-0.05*si) - math.exp(-0.05*ei)) / 0.05
        SD.append(sd)

In [66]:
# middle_df_generator(original_df) generates the middle_df for further cal
def middle_df_generator(original_df):
    data = {}
    columns = original_df.columns.tolist()
    
    hedging_set = original_df['Base currency'].tolist()
    nature_set = original_df['Nature'].tolist()
    length = len(nature_set)
    
    for i in range(0, length):
        indicator = nature_classifier(original_df, i)
        if indicator == 2:
            hedging_set[i] = 'CDS'
    Ei = []
    Si = []
    residual_maturity_converter(Ei, Si, original_df['Residual maturity'])
    SD = []
    SD_generator(Si, Ei, SD)
    Adj_notion = np.array(SD) * np.array(original_df['Notional (thousands)'].tolist())
    
    data.update({"Hedging set": hedging_set,
                 "Notional (thousands)": original_df['Notional (thousands)'].tolist(),
                "Si": Si, 
                "Ei":Ei,
                "SDi": SD,
                "Adjusted notional (thousands)": Adj_notion.tolist(),             
                })
    
    classes = []
    for number in Ei:
        if number > 5:
            classes.append(3)
        elif number < 1:
            classes.append(1)
        else:
            classes.append(2)
    data.update({'Time Bucket': classes})
    
    if 'Reference entity / index name' in columns:
        entity_set = original_df['Reference entity / index name'].tolist()
        data.update({'Reference entity / index name': entity_set})
    #print(data)
    return pd.DataFrame(data)

In [67]:
# phi() takes in a number and calculate the phi of it
def phi(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

In [150]:
# supervisory_delta_generator() takes in two dataframe, the original one
# and the new generated one to modify and add a column on the 
# new generated dataframe
def supervisory_delta_generator(original_df, middle_df):
    (row_num, col_num) = original_df.shape
    sd_column = []
    for i in range(0, row_num):
        sd = None
        indicator = nature_classifier(original_df, i)
        if (indicator == 1):
        # This happens if it is a interest rate derivative
            fixed = (str(original_df.at[i, 'Pay Leg (*)']) == 'Fixed')
            option_idx = (str(original_df.at[i, 'Nature']).find("swaption"))
        
            if (fixed and option_idx == -1): # long but not option
                sd = 1
            elif (not fixed and option_idx == -1): # short but not option
                sd = -1
            else: # option
                symbol1 = 1
                symbol2 = 1
                if not fixed: # put option
                    symbol2 = -1
                if (original_df.at[i, 'Option position'] == 'Bought' and not fixed) or original_df.at[i, 'Option position'] != 'Bought' and fixed: #if it is call option
                    symbol1 = -1
                Ti = int(middle_df.at[i, 'Si'])
                underlying_price = original_df.at[i, 'Underlying price']
                strike_price = original_df.at[i, 'Strike price']
                sd = symbol1 * phi(symbol2 * 
                                  (math.log(underlying_price / strike_price) + 
                                  0.5 * 0.5 ** 2 * Ti) / 
                                  (0.5 * math.sqrt(Ti)))
        
        if (indicator == 2): # if it is a credit derivative not an option
            #print('case two')
            long = str(original_df.at[i, 'Position']).find('buyer')
            if (long != -1): # it is a buyer so it would be long
                sd = 1
            else:
                sd = -1
        sd_column.append(sd)
        
    middle_df['Supervisory delta'] = sd_column

In [151]:
# Hedging list generator
def hedging_list_generator(middle_df):
    
    #Hedging list only works for credit deriviative
    hedging_set = []
    (row_num, col_num) = middle_df.shape
    
    for i in range(0, row_num):
            if middle_df.at[i, 'Hedging set'] != 'CDS':
                
                series1 = middle_df.at[i, 'Adjusted notional (thousands)']
                series2 = middle_df.at[i, 'Supervisory delta']
                hedging_s = series1 * series2
                hedging_set.append(hedging_s)
            else:
                hedging_set.append(None)
                
    return hedging_set

In [323]:
# EffectiveNotional generator for interest rate OR addOn value for 
def effectiveNotional(original_df, middle_df, group_name, indicator):
    
    key_list = []
    hedging_set = hedging_list_generator(middle_df)
    #print("Hedging set from effectiveNotional function: ", hedging_set)
    
    #entity reference for solely credit derivative calculation
    if 'Reference entity / index name' in original_df.columns.tolist(): 
        entity_set = [ x for x in list(set(middle_df['Reference entity / index name'])) if str(x) != 'nan']
        r_list = {}
        modified_r_list = {}
    # initialization
        entity_addOn_dict = {}
        for entity in entity_set:
            entity_addOn_dict.update({entity: 0})
            if entity not in key_list:
                key_list.append(entity)   
        
    (row_num, col_num) = original_df.shape
    en_list = [0, 0, 0]
    idx = 0
    for i in range(0,row_num): 
        if (indicator == 1 and str(middle_df.at[i, 'Hedging set']) == str(group_name)):
            idx = middle_df.at[i, 'Time Bucket'] - 1
            en_list[idx] = hedging_set[i]
            
            
        elif (indicator == 2 and str(middle_df.at[i, 'Hedging set']) == str(group_name)):
            
            key = str(middle_df.at[i, 'Reference entity / index name'])
            symbol = int(middle_df.at[i, 'Supervisory delta'])
            rating_ref = str(original_df.at[i, 'Rating reference entity'])
            #|print(rating_ref)
            if rating_ref == "Investment grade":
                rating_ref = 'IG'
            
            sf = pd.DataFrame(supervisory_ref[supervisory_ref['Subclass'] == rating_ref])['Supervisory factor'].tolist()[0]
            adj_notion = float(middle_df.at[i, 'Adjusted notional (thousands)'])
            #print(symbol, sf, adj_notion)
            addOn_temp = adj_notion * sf * symbol
            #print(addOn_temp)
            r = pd.DataFrame(supervisory_ref[supervisory_ref['Subclass'] == rating_ref])['Correlation'].tolist()[0]
            modified_r = 1 - r ** 2
            
            entity_addOn_dict.update({key: entity_addOn_dict.get(key) + addOn_temp})
            if key not in r_list.keys():
                r_list.update({key: r})
                modified_r_list.update({key: modified_r})
            
            
    #print("en list: ", en_list)
    if (indicator == 1):
        if None not in en_list:
            e_n = 0
            for item in en_list:
            #if item != None:
                e_n += item ** 2
        #print(e_n)
            e_n += 1.4 * (en_list[0]) * (en_list[1])
            e_n += 1.4 * (en_list[1]) * (en_list[2])
            e_n += 0.6 * (en_list[0]) * (en_list[2])
        #print(e_n)
            result = math.sqrt(e_n)
            return result
    
    
    elif (indicator == 2):
        addOn_value_list = []
        for entity in key_list:
            addOn_value_list.append(entity_addOn_dict.get(entity))
        
        entity_level_add_on_times_r = []
        entity_level_add_on_times_one_m = []
        
        for entity in key_list:
            add_on_value = entity_addOn_dict.get(entity)
            r_value = r_list.get(entity)
            m_r = modified_r_list.get(entity)
            
            ent_add_on_t_r = add_on_value * r_value
            ent_add_on_t_one_m = add_on_value * add_on_value * m_r
            
            entity_level_add_on_times_r.append(ent_add_on_t_r)
            entity_level_add_on_times_one_m.append(ent_add_on_t_one_m)
#         entity_level_add_on_times_r = np.array(addOn_value_list) * np.array(r_list)
#         entity_level_add_on_times_one_m = np.square(np.array(addOn_value_list)) * np.array(modified_r_list)
            
        print('Add on value keys list:', list(entity_addOn_dict.keys()))
        print('Add On value list:', addOn_value_list)
        print("r list: ", r_list.values())
        print("1 - r^2:", modified_r_list.values())
#        print(entity_level_add_on_times_r)
#         print(entity_level_add_on_times_one_m)
        
        addOn_credit = np.sum(entity_level_add_on_times_r) ** 2 + np.sum(entity_level_add_on_times_one_m)
        addOn_credit = math.sqrt(addOn_credit)
        #print(addOn_credit)
        return addOn_credit

In [324]:
# addOn calculator
def addOn_generator(original_df, middle_df, supervisory_ref):
    addOn = []
    unique_group = list(set(middle_df['Hedging set'].tolist()))
    #print(unique_group)
    CDS = False
    interest_rate = False
    for group in unique_group:
        if (group.find("CDS") != -1):
            CDS = True
            #print("CDS exists")
            break
    if CDS:
        for group in unique_group:
            if group.find("CDS") != -1:
                unique_group.remove(group)
                
        unique_group.append('CDS')
        
    effectiveNotionList_interest = []
    entity_level_add_on_times_r = []
    # = []
    entity_level_add_on_times_one_m = []
    #print("addOn func unique groups: ", unique_group)
    #i = 0
    (row_num, col_num) = original_df.shape

    #for i in range(0, row_num):
    for hedging_set in unique_group: # e.g.['USD', 'EUR', 'CDS']
        #for i in range(1, 3):
        
        #indicator = nature_classifier(original_df, i)
        indicator = 0
        if hedging_set in ['USD', 'EUR']:
            indicator = 1
        elif hedging_set in ['CDS']:
            indicator = 2
        #print(indicator)
        if indicator == 1: # deal with cases of interest rate
            e_n_i = effectiveNotional(original_df, middle_df, hedging_set, indicator)
            #print('ENI: ', e_n_i)
            if e_n_i != None:
                effectiveNotionList_interest.append(e_n_i)
            interest_rate = True;
            
            
        elif indicator == 2: #deal with cases of CDS
            print("addOn indicator = 2: ", effectiveNotional(original_df, middle_df, hedging_set, 2))
            addOn.append(effectiveNotional(original_df, middle_df,hedging_set, 2))
            #print("Test:", (effectiveNotional(original_df, middle_df,hedging_set, 2)))
            #print(addOn)
            CDS = True
            
        #i += 1
    #print(effectiveNotionList)
   
    if (interest_rate):
        newList = 0.005 * np.array(effectiveNotionList_interest)
        addOn_interest = np.sum(newList)
        addOn.append(addOn_interest)
    
    # we now deal with the credit part
    #if (CDS):
    #    print("CDS IS TRUE")
    #if (interest_rate):
    #    print("interest rate is TRUE")
    print("addOn: ", addOn)
    return (np.sum(addOn))

In [325]:
# find the multipler for interest rate EAD
def multiplier_finder(original_df, middle_df):
    addOn = addOn_generator(original_df, middle_df, supervisory_ref)
    floor = 0.05
    v_c = original_df[['Market value (thousands)']].sum()
    denominator = 2 * (1 - floor) * addOn
    alternative = floor + (1 - floor) * math.exp(v_c / denominator)
    multiplier = min(1, alternative)
    return multiplier

In [326]:
# EAD for interest rate
def EAD_generator(original_df, middle_df, supervisory_ref):
    r_c = rc(original_df)
    print("RC:", r_c)
    multiplier = multiplier_finder(original_df, middle_df)
    print('multiplier:', multiplier)
    addon_val = addOn_generator(original_df, middle_df, supervisory_ref)
    print("addOn Value", addon_val)
    
    return 1.4 * (r_c + multiplier * addon_val)

# Example 1

In [327]:
example_1_df = pd.read_csv('example_1_input.csv')
example_1_df

,Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands),Option position,Underlying price,Strike price
0,1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30,NaN,NaN,NaN
1,2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20,NaN,NaN,NaN
2,3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50,Bought,0.06,0.05


In [351]:
example_1_df = pd.read_csv('example_1_input.csv')
middle_df = middle_df_generator(example_1_df)
supervisory_delta_generator(example_1_df, middle_df)
#time_bucket_classifier(example_1_df, middle_df)
hedging_list_generator(middle_df)
addOn_generator(example_1_df, middle_df,supervisory_ref)
middle_df

[10.0, 4.0, 11.0] [0, 0, 1] 0           10 years
1            4 years
2    1 into 10 years
Name: Residual maturity, dtype: object
addOn:  [346.7643863838184]


,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Supervisory delta
0,USD,10000,0,10.0,7.869387,78693.868057,3,1.000000
1,USD,10000,0,4.0,3.625385,36253.849384,2,-1.000000
2,EUR,5000,1,11.0,7.485592,37427.961412,3,-0.269395


In [329]:
EAD_generator(example_1_df, middle_df, supervisory_ref)

RC: 60
addOn:  [346.7643863838184]
multiplier: 1
addOn:  [346.7643863838184]
addOn Value 346.7643863838184


569.4701409373457

In [330]:
middle_df

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Supervisory delta
0,USD,10000,0,10.0,7.869387,78693.868057,3,1.000000
1,USD,10000,0,4.0,3.625385,36253.849384,2,-1.000000
2,EUR,5000,1,11.0,7.485592,37427.961412,3,-0.269395


# Example 2

In [331]:
example_2_input = pd.read_csv('example_2_input.csv')
example_2_input

,Trade #,Nature,Reference entity / index name,Rating reference entity,Residual maturity,Base currency,Notional (thousands),Position,Market value (thousands)
0,1,Single-name CDS,Firm A,AA,3 years,USD,10000,Protection buyer,20
1,2,Single-name CDS,Firm B,BBB,6 years,EUR,10000,Protection seller,-40
2,3,CDS index,CDX.IG 5y,Investment grade,5 years,USD,10000,Protection buyer,0


In [332]:
ex_2_middle = middle_df_generator(example_2_input)
ex_2_middle

[3.0, 6.0, 5.0] [0, 0, 0] 0    3 years
1    6 years
2    5 years
Name: Residual maturity, dtype: object


,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Reference entity / index name
0,CDS,10000,0,3.0,2.785840,27858.404715,2,Firm A
1,CDS,10000,0,6.0,5.183636,51836.355864,3,Firm B
2,CDS,10000,0,5.0,4.423984,44239.843386,2,CDX.IG 5y


In [333]:
supervisory_delta_generator(example_2_input, ex_2_middle)
ex_2_middle

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Reference entity / index name,Supervisory delta
0,CDS,10000,0,3.0,2.785840,27858.404715,2,Firm A,1
1,CDS,10000,0,6.0,5.183636,51836.355864,3,Firm B,-1
2,CDS,10000,0,5.0,4.423984,44239.843386,2,CDX.IG 5y,1


In [334]:
addOn_generator(example_2_input, ex_2_middle,supervisory_ref)

Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn:  [282.1288318596666]


282.1288318596666

In [335]:
EAD_generator(example_2_input, ex_2_middle,supervisory_ref)

RC: 0
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn:  [282.1288318596666]
multiplier: 0.965208280997999
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values(

381.2383187469392

# Example 4

In [343]:
example_4_input = pd.read_csv('example_4_input.csv')
example_4_input

,Trade #,Nature,Residual maturity,Base currency,Notional (thousands),Pay Leg (*),Receive Leg (*),Market value (thousands),Reference entity / index name,Rating reference entity,Position,Option position,Underlying price,Strike price
0,1,Interest rate swap,10 years,USD,10000,Fixed,Floating,30,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Interest rate swap,4 years,USD,10000,Floating,Fixed,-20,NaN,NaN,NaN,NaN,NaN,NaN
2,3,European swaption,1 into 10 years,EUR,5000,Floating,Fixed,50,NaN,NaN,NaN,Bought,0.06,0.05
3,4,Single-name CDS,3 years,USD,10000,NaN,NaN,20,Firm A,AA,Protection buyer,NaN,NaN,NaN
4,5,Single-name CDS,6 years,EUR,10000,NaN,NaN,-40,Firm B,BBB,Protection seller,NaN,NaN,NaN
5,6,CDS index,5 years,USD,10000,NaN,NaN,0,CDX.IG 5y,Investment grade,Protection buyer,NaN,NaN,NaN


In [344]:
example_4_input.columns

Index(['Trade #', 'Nature', 'Residual maturity', 'Base currency',
       'Notional (thousands)', 'Pay Leg (*)', 'Receive Leg (*)',
       'Market value (thousands)', 'Reference entity / index name',
       'Rating reference entity', 'Position', 'Option position',
       'Underlying price', 'Strike price'],
      dtype='object')

In [345]:
ex_4_middle = middle_df_generator(example_4_input)
ex_4_middle

[10.0, 4.0, 11.0, 3.0, 6.0, 5.0] [0, 0, 1, 0, 0, 0] 0           10 years
1            4 years
2    1 into 10 years
3            3 years
4            6 years
5            5 years
Name: Residual maturity, dtype: object


,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Reference entity / index name
0,USD,10000,0,10.0,7.869387,78693.868057,3,NaN
1,USD,10000,0,4.0,3.625385,36253.849384,2,NaN
2,EUR,5000,1,11.0,7.485592,37427.961412,3,NaN
3,CDS,10000,0,3.0,2.785840,27858.404715,2,Firm A
4,CDS,10000,0,6.0,5.183636,51836.355864,3,Firm B
5,CDS,10000,0,5.0,4.423984,44239.843386,2,CDX.IG 5y


In [346]:
supervisory_delta_generator(example_4_input, ex_4_middle)
ex_4_middle

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Time Bucket,Reference entity / index name,Supervisory delta
0,USD,10000,0,10.0,7.869387,78693.868057,3,NaN,1.000000
1,USD,10000,0,4.0,3.625385,36253.849384,2,NaN,-1.000000
2,EUR,5000,1,11.0,7.485592,37427.961412,3,NaN,-0.269395
3,CDS,10000,0,3.0,2.785840,27858.404715,2,Firm A,1.000000
4,CDS,10000,0,6.0,5.183636,51836.355864,3,Firm B,-1.000000
5,CDS,10000,0,5.0,4.423984,44239.843386,2,CDX.IG 5y,1.000000


In [347]:
rc(example_4_input)

40

In [348]:
addOn_generator(example_4_input, ex_4_middle, supervisory_ref)

Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn:  [282.1288318596666, 346.7643863838184]


628.8932182434851

In [349]:
EAD_generator(example_4_input, ex_4_middle,supervisory_ref)

RC: 40
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn:  [282.1288318596666, 346.7643863838184]
multiplier: 1
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_values([0.5, 0.5, 0.8])
1 - r^2: dict_values([0.75, 0.75, 0.3599999999999999])
addOn indicator = 2:  282.1288318596666
Add on value keys list: ['Firm B', 'CDX.IG 5y', 'Firm A']
Add On value list: [-279.91632166374467, 168.11140486573225, 105.86193791695607]
r list:  dict_val

936.450505540879